In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Importing necessary modules
import keras
from keras.layers import Conv2D,Dense,MaxPooling2D,Flatten,Dropout

In [ ]:
#Loading the train_dataset with train.csv
train_dataset=pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [ ]:
#Mkaing a numpy array of labels 
labels=train_dataset.label.to_numpy()
print("Labels : ",labels)

In [ ]:
#Converting class vector (integers) into binary class matrix
labels=keras.utils.to_categorical(labels,num_classes=10)
print("Binary Class Matrix : ",labels)

In [ ]:
#Save all the columns other than the label column in train_images dataset
train_images=train_dataset[[col for col in train_dataset.columns if col!='label']]

In [ ]:
#Printing the shape of train_iamges
train_images.shape

In [ ]:
"""Input : A batch of images of the form (28,28)
Functionality : Pre process the image data to convert the input into (batch_size,28,28,1)
Returns : Processed batch of images of the shape (batch_size,28,28,1)
"""
def pre_processing(data):
    #Copying the data in numpy array to a variable
    copy_array=data.to_numpy(copy=True)
    #matrix of shape (batch_size,28,28) containing zeros
    images_train=np.zeros((data.shape[0],28,28),dtype='float32')
    #declaring variable of training examples
    training_examples=images_train.shape[0]
    #for i in the range of training examples (running a loop process on the batch)
    for i in range(training_examples):
        #declaring temp variable to reshaped matrix of copied array to (28,28) 
        temp=np.reshape(copy_array[i],(28,28))
        #Assigning the variabe images_train to temp
        images_train[i]=temp
    #Rescaling all the iamges on the scale of 0 to 1
    images_train=images_train[:]/255.0
    #Reshaping the iamges_train to (batch_size,28,28,1)
    images_train=np.reshape(images_train,(data.shape[0],28,28,1))
    #Returning the preprocessed training image data
    return images_train
#Calling the function by passing the batch of images 
images_train=pre_processing(train_images)
#Printing the shape of processed image data
print("Shape of training data : ",images_train.shape)
    

In [ ]:
X=keras.models.Sequential() #Initializing the sequential model
X.add(Conv2D(32,(3,2),activation='relu',input_shape=(28,28,1)))
X.add(Conv2D(64,(3,3),activation='relu'))
X.add(MaxPooling2D((3,3),padding='same'))
X.add(Conv2D(64,(3,3),activation='relu'))
X.add(MaxPooling2D((3,3),padding='same'))
X.add(Flatten())
X.add(Dense(64))
X.add(Dropout(0.2))
X.add(Dense(10,activation='softmax'))

      
      


In [ ]:
print(X.summary())

In [ ]:
X.compile(optimizer='adam',loss="categorical_crossentropy",metrics=['accuracy'])


In [ ]:
X.fit(images_train,labels,epochs=15,batch_size=64)

In [ ]:
test_dataset=pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
test_dataset.head()

In [ ]:
test_images=pre_processing(test_dataset)
predictions=X.predict(test_images)

In [ ]:
#predicts is an array with 10 columns but we want just with higher probability 
results=np.argmax(predictions,axis=1)
print(results.shape)

In [ ]:
#Converting the array into a dataframe
results_df=pd.DataFrame(results)

In [ ]:
results_df.columns=['Label']
results_df['ImageId']=[i for i in range(1,results.shape[0]+1)]
results_df=results_df[['ImageId','Label']]

In [ ]:
print(results_df)

In [ ]:
results_df.to_csv('Digit Prediction',index=False)